In [5]:
from numpy import sqrt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


ModuleNotFoundError: No module named 'tensorflow.python'

In [ ]:
path = "D:\MachineLearning\data\ParisHousingClass.csv"
data = pd.read_csv(path)
data.replace(['NaN', 'N/A', 'NA', 'n/a', 'n.a.', 'N#A', 'n#a', '?'], 'other', inplace=True)
data.head()
data.info()

In [ ]:
numerical_vars = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_vars = data.select_dtypes(include=['object']).columns.tolist()                           
print('Numerical variables:', numerical_vars)
print('Categorical variables:', categorical_vars)
print(data.dtypes)

In [ ]:
categorical_count = data.select_dtypes(include='object').shape[1]
numerical_count = data.select_dtypes(exclude='object').shape[1]

print(f"Biến phân loại: {categorical_count}")
print(f"Biến không phân loại: {numerical_count}")
print(data.select_dtypes(include=['object']).nunique())

In [ ]:
catcol = []
numcol = []

for col in data.columns:
    if data[col].dtype == 'object':
        catcol.append(col)
    else:
        numcol.append(col)
print("Categorical Columns:",catcol)
print("Numerical Columns:", numcol)

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

for col in catcol:
    data[col] = encoder.fit_transform(data[col])
data.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

cols_to_scale = [col for col in numcol if col != "price"]

scale = MinMaxScaler()

for col in cols_to_scale:
    #if col != 'price':
    data[[col]] = scale.fit_transform(data[[col]])
data.head()

In [ ]:
X = data.drop(columns=['price'])
y = data['price']
X.head(), y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42)
X_train.head(), X_test.head(), y_train.head(), y_test.head()

In [ ]:
n_features = X_train.shape[1]
n_features

In [ ]:
model = Sequential()

model.add(Dense(64, activation ='relu', kernel_initializer='he_normal', input_shape = (n_features,)))

model.add(Dense(32, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(16, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))

model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

history = model.fit(X_train, y_train, epochs=100, batch_size=4, verbose=2)

In [ ]:
error = model.evaluate(X_test, y_test, verbose=2)
print('MSE: %.3f' % (error))
print('RMSE: %.3f' % (sqrt(error)))

In [ ]:
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
y_test, y_pred
#print('R²: %.3f' % r2)

In [ ]:
import pickle #dump or save model
file_name = 'mlp'
with open(file_name, 'wb') as file: 
    pickle.dump(model, file)

In [ ]:

plt.figure(figsize=(10, 6))

plt.scatter(y_test, y_pred, color='blue', alpha=0.6, label='Predicted vs Actual')


plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linewidth=2, label='Perfect Prediction')

plt.title('Actual vs Predicted Values')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.grid()

plt.show()